<a href="https://colab.research.google.com/github/minzzii-kim/machine-learing/blob/main/Copy_of_tensorflow_note_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# [이미지 분류기 2]
# Type B : tfds 사용
# cats vs dogs 이진분류

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout,Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

In [3]:
# tfds 데이터 전처리

dataset_name = 'cats_vs_dogs'
train_ds = tfds.load(name=dataset_name, split='train[:80%]')
valid_ds = tfds.load(name=dataset_name, split='train[80%:]')

for data in train_ds.take(3):
  #print(data['label'])
  y=data['label']
  
def preprocess(data):
  x=data['image']
  y=data['label']

  x=x/255
  x=tf.image.resize(x,size=(224,224))
  return x,y

BATCH_SIZE=32
train_data=train_ds.map(preprocess).batch(BATCH_SIZE)
valid_data=valid_ds.map(preprocess).batch(BATCH_SIZE)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/cats_vs_dogs/4.0.0.incompleteATPHZ6/cats_vs_dogs-train.tfrecord


  0%|          | 0/23262 [00:00<?, ? examples/s]

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.


In [4]:
# 모델 정의 및 생성
model = Sequential([
  Conv2D(64, (3,3), activation='relu', input_shape=(224,224,3)),
  MaxPooling2D(2,2),
  Conv2D(64, (3,3), activation='relu'),
  MaxPooling2D(2,2),
  Conv2D(128, (3,3), activation='relu'),
  MaxPooling2D(2,2),
  Conv2D(128, (3,3), activation='relu'),
  MaxPooling2D(2,2),
  Flatten(),
  Dropout(0.5),
  Dense(64, activation='relu'),
  Dense(32, activation='relu'),
  Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 64)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 26, 26, 128)      0

In [6]:
tmp_path = 'my_checkpoint.ckpt'
checkpoint = ModelCheckpoint(tmp_path,monitor='val_loss',save_best_only=True,save_weights_only=True,verbose=1)

model.fit(train_data, validation_data=(valid_data), epochs=25, callbacks=[checkpoint])

model.load_weights(tmp_path)
model.evaluate(valid_data)

Epoch 1/25
582/582 [==============================] - ETA: 0s - loss: 0.6772 - acc: 0.5620
Epoch 00001: val_loss improved from inf to 0.62741, saving model to my_checkpoint.ckpt
582/582 [==============================] - 100s 149ms/step - loss: 0.6772 - acc: 0.5620 - val_loss: 0.6274 - val_acc: 0.6563
Epoch 2/25
582/582 [==============================] - ETA: 0s - loss: 0.5765 - acc: 0.6962
Epoch 00002: val_loss improved from 0.62741 to 0.52709, saving model to my_checkpoint.ckpt
582/582 [==============================] - 85s 146ms/step - loss: 0.5765 - acc: 0.6962 - val_loss: 0.5271 - val_acc: 0.7498
Epoch 3/25
582/582 [==============================] - ETA: 0s - loss: 0.4914 - acc: 0.7629
Epoch 00003: val_loss improved from 0.52709 to 0.43951, saving model to my_checkpoint.ckpt
582/582 [==============================] - 85s 146ms/step - loss: 0.4914 - acc: 0.7629 - val_loss: 0.4395 - val_acc: 0.8050
Epoch 4/25
582/582 [==============================] - ETA: 0s - loss: 0.4254 - acc: 0

[0.3453195095062256, 0.8561908602714539]

In [6]:
# VGG16 과 같은 전이학습 모델을 사용할 경우,
# 성능 loss, acc = [0.3453195095062256, 0.8561908602714539]